# Browse experiment results ...

In [1]:
import torch
import pickle
import glob
import os.path
import pandas as pd
from train_engine import __training_cfg, __exp_res_meta
import numpy as np
# from core.utils_ipynb import read_exp_result_files
import os

def read_exp_result_files(path):
    files = glob.glob(os.path.join(path, "*.pickle"))
    res = []
    for f in files: 
        if os.path.basename(f) == 'errors.pickle':
            continue        
        
        r = pickle.load(open(f, 'rb'))
        
        #older cfgs have no 'set_node_degree_uninformative' ... 
        if 'set_node_degree_uninformative' not in r['exp_cfg']['model']:
            r['exp_cfg']['model']['set_node_degree_uninformative'] = False
            
        res.append(r)
    return res


def get_keychain_value_iter(d, key_chain=None):
    key_chain = [] if key_chain is None else list(key_chain).copy()       
    
    if not isinstance(d, dict):
        
        yield tuple(key_chain), d
    else:
        for k, v in d.items():
            yield from get_keychain_value_iter(v, key_chain + [k])
            
def get_keychain_value(d, key_chain):
    
    try:
        
        for k in key_chain:
            d = d[k]
            
    except Exception as ex:
        raise KeyError() from ex
        
    return d

In [2]:
kc = {k: k[-1] for k, v in list(get_keychain_value_iter(__exp_res_meta))}
kc

{('exp_cfg', 'dataset_name'): 'dataset_name',
 ('exp_cfg', 'training', 'lr'): 'lr',
 ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
 ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
 ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
 ('exp_cfg', 'training', 'batch_size'): 'batch_size',
 ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
 ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
 ('exp_cfg', 'model', 'model_type'): 'model_type',
 ('exp_cfg',
  'model',
  'use_super_level_set_filtration'): 'use_super_level_set_filtration',
 ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
 ('exp_cfg',
  'model',
  'set_node_degree_uninformative'): 'set_node_degree_uninformative',
 ('exp_cfg', 'model', 'pooling_strategy'): 'pooling_strategy',
 ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
 ('exp_cfg', 'model', 'gin_number'): 'gin_number',
 ('exp_cfg', 'model', 'gin_dimension'): 'gin_dimension',
 ('exp_cfg', 'model', 'gin_mlp_type'): 'gi

In [3]:
COL_NAMES = {
    ('exp_cfg', 'dataset_name'): 'dataset_name',
    #('exp_cfg', 'tag'): 'tag', 
#     ('exp_cfg', 'training', 'lr'): 'lr',
#     ('exp_cfg', 'training', 'lr_drop_fact'): 'lr_drop_fact',
#     ('exp_cfg', 'training', 'num_epochs'): 'num_epochs',
#     ('exp_cfg', 'training', 'epoch_step'): 'epoch_step',
    # ('exp_cfg', 'training', 'batch_size'): 'batch_size',
#     ('exp_cfg', 'training', 'weight_decay'): 'weight_decay',
#     ('exp_cfg', 'training', 'validation_ratio'): 'validation_ratio',
    ('exp_cfg', 'model', 'model_type'): 'model_type',
    # ('exp_cfg', 'model', 'use_super_level_set_filtration'): 'use_super_level_set_filtration',
    # ('exp_cfg', 'model', 'use_node_degree'): 'use_node_degree',
    # ('exp_cfg', 'model', 'use_node_label'): 'use_node_label',
    # ('exp_cfg', 'model', 'gin_number'): 'gin_number',
    # ('exp_cfg', 'model', 'cls_hidden_dimension'): 'cls_hidden_dimension',
    #('exp_cfg', 'model', 'gin_mlp_type'): 'gin_mlp_type',
    # ('exp_cfg', 'model', 'set_node_degree_uninformative'): 'set_node_degree_uninformative',
    # ('exp_cfg', 'model', 'num_struct_elements'): 'num_struct_elements',
    # ('exp_cfg', 'model', 'drop_out'): 'drop_out',
    # ('exp_cfg', 'model', 'pooling_strategy'): 'pooling_strategy',
    ('exp_cfg', 'model', 'concat_fp'): 'fp',
    # ('cv_test_acc',): 'cv_test_acc',
#     ('cv_val_acc',): 'cv_val_acc',
#     ('cv_indices_trn_tst_val',): 'cv_indices_trn_tst_val',
#     ('cv_epoch_loss',): 'cv_epoch_loss',
#     ('start_time',): 'start_time',
#     ('id',): 'id',
    ('finished_training',): 'finished_training'
}

In [4]:
def pd_frame(path):
    
    f = read_exp_result_files(path)
    
    data_frames = []
    for i, res in enumerate(f):
        row = {}
        # print(res.keys())
        cv_acc_last = [x[-1] for x in res['cv_test_acc'] if len(x) > 0]
        cv_roc_last = []
        if 'cv_test_roc' in res.keys():
            cv_roc_last = [x[-1] for x in res['cv_test_roc'] if len(x) > 0]
        
        row['acc_last_mean'] = np.mean(cv_acc_last)
        row['acc_last_std'] = np.std(cv_acc_last)
        row['roc_last_mean'] = np.mean(cv_roc_last) if len(cv_roc_last) > 0 else np.nan
        row['roc_last_std'] = np.std(cv_roc_last) if len(cv_roc_last) > 0 else np.nan
        row['start_time'] = res['start_time']
        
        cv_acc_validated = []
        for test, val in zip(res['cv_test_acc'], res['cv_val_acc']):
            if not len(test) == res['exp_cfg']['training']['num_epochs']:
                continue
            # n = len(test)//2
            # test = torch.tensor(test[n:])
            # val = torch.tensor(val[n:])
            test = torch.tensor(test)
            val = torch.tensor(val)
            
            
            _, i_max = val.max(0)
            cv_acc_validated.append(test[i_max].item())
        if 'cv_test_roc' in res.keys():
            cv_roc_validated = []
            for test, val in zip(res['cv_test_roc'], res['cv_val_roc']):
                if not len(test) == res['exp_cfg']['training']['num_epochs']:
                    continue
                # n = len(test)//2
                # test = torch.tensor(test[n:])
                # val = torch.tensor(val[n:])
                test = torch.tensor(test)
                val = torch.tensor(val)
                
                
                _, i_max = val.max(0)
                cv_roc_validated.append(test[i_max].item())
                
            row['roc_val_mean'] = np.mean(cv_roc_validated)
            row['roc_val_std'] = np.std(cv_roc_validated)
        else:
            row['roc_val_mean'] = np.nan
            row['roc_val_std'] = np.nan
        
        
        cv_folds_available = sum([1 for cv in res['cv_test_acc'] if len(cv) == res['exp_cfg']['training']['num_epochs']])
        row['cv_folds_available'] = cv_folds_available
        
        
        for k, v in COL_NAMES.items():
            try:
                row[v] = get_keychain_value(res, k)
            except KeyError:
                pass

        f = pd.DataFrame(row, index=[i])
        data_frames.append(f)
        
    return pd.concat(data_frames, sort=True)

In [5]:
path = './experiment_logs/'
RES = pd_frame(path)
RES = RES.sort_values(by=['start_time','dataset_name', 'model_type'], ascending=False)
# RES = RES[RES['dataset_name'] == 'EGFR']
RES = RES[(RES['model_type'] == 'GIN_MPML') & RES['dataset_name'].str.contains('CHEMBL')]
# RES = RES[(RES['model_type'] == 'SimpleNNBaseline')]
# RES = RES[RES['fp'] == 'morgan3']
# RES = RES[RES['dataset_name'].str.contains('CHEMBL')]
# RES = RES[RES['dataset_name'].str.contains('ERRB2')]
# RES[RES['dataset_name'].str.contains('REDDIT') & (RES['gin_number'] == 3)]
# RES = RES[(RES['cv_folds_available'] == 5) & (RES['model_type'] == 'GIN_MPML')]
RES

,acc_last_mean,acc_last_std,cv_folds_available,dataset_name,finished_training,fp,model_type,roc_last_mean,roc_last_std,roc_val_mean,roc_val_std,start_time
85,81.554571,0.724010,4,CHEMBL279,False,morgan3,GIN_MPML,77.652019,0.860329,78.204943,0.312386,2024-01-24 23:31:09.057389
127,82.438149,0.823589,5,CHEMBL203,True,morgan3,GIN_MPML,81.028462,0.925093,81.093901,0.493027,2024-01-21 21:26:02.705568
25,83.202847,2.515387,5,CHEMBL267,True,morgan3,GIN_MPML,83.031341,2.520094,82.791311,2.450759,2024-01-20 16:52:02.234401
48,83.758550,0.973337,5,CHEMBL203,True,morgan2,GIN_MPML,82.174309,1.366117,82.167550,0.699985,2024-01-16 12:12:10.684405
56,82.027290,0.942148,5,CHEMBL4722,True,ecfp,GIN_MPML,77.650657,1.874811,78.030500,1.368618,2024-01-14 14:31:39.446892
71,83.088686,1.264352,5,CHEMBL4282,True,ecfp,GIN_MPML,78.388606,1.701071,78.915437,2.097622,2024-01-13 00:02:22.021968
99,73.138402,2.165059,5,CHEMBL4722,True,maccs,GIN_MPML,62.253462,3.111438,64.198247,1.394435,2024-01-12 23:01:03.485049
89,74.733389,3.295665,5,CHEMBL4282,True,maccs,GIN_MPML,68.020460,4.440043,71.333285,1.973594,2024-01-11 11:48:16.103171
98,83.131524,1.276051,5,CHEMBL4005,True,ecfp,GIN_MPML,80.447898,1.400009,81.592516,1.580912,2024-01-10 11:15:01.519542
81,72.651357,1.395784,5,CHEMBL4005,True,maccs,GIN_MPML,65.841514,3.317140,71.989982,2.385616,2024-01-09 03:14:04.543264


In [7]:
print(len(RES))
# RES.to_csv('results_gin.csv')

27


The following cells contain some utility for messing around with results, i.e., deleting etc. 

In [8]:
# num_completed = len(RES[(RES['model_type'] == 'GIN') & (RES['dataset_name'].str.contains('CHEMBL')) & (RES['fp'].str.contains('morgan2'))])
# num_completed